In [155]:
# %load app.py
# import necessary libraries
import os
from flask import (
    Flask,
    render_template,
    jsonify,
    request,
    redirect)
from sqlalchemy import MetaData
from sqlalchemy import inspect
from sqlalchemy import Table

import pandas as pd

In [22]:
#################################################
# Flask Setup
#################################################
app = Flask(__name__)

from flask_sqlalchemy import SQLAlchemy
app.config['SQLALCHEMY_DATABASE_URI'] = "sqlite:///Datasets/belly_button_biodiversity.sqlite"
db = SQLAlchemy(app)

C:\ProgramData\Anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


# TESTING

In [16]:
db.Model.metadata.reflect(db.engine)

In [64]:
print(db.engine.table_names())

['otu', 'samples', 'samples_metadata']


In [83]:
# got metadata to work
metadata = MetaData()
metadata.reflect(bind=db.engine)

In [85]:
for t in metadata.tables:
    print("table", t)

table otu
table samples
table samples_metadata


In [111]:
samples = metadata.tables['samples']
samples

Table('samples', MetaData(bind=None), Column('otu_id', INTEGER(), table=<samples>, primary_key=True, nullable=False), Column('BB_940', INTEGER(), table=<samples>), Column('BB_941', INTEGER(), table=<samples>), Column('BB_943', INTEGER(), table=<samples>), Column('BB_944', INTEGER(), table=<samples>), Column('BB_945', INTEGER(), table=<samples>), Column('BB_946', INTEGER(), table=<samples>), Column('BB_947', INTEGER(), table=<samples>), Column('BB_948', INTEGER(), table=<samples>), Column('BB_949', INTEGER(), table=<samples>), Column('BB_950', INTEGER(), table=<samples>), Column('BB_952', INTEGER(), table=<samples>), Column('BB_953', INTEGER(), table=<samples>), Column('BB_954', INTEGER(), table=<samples>), Column('BB_955', INTEGER(), table=<samples>), Column('BB_956', INTEGER(), table=<samples>), Column('BB_958', INTEGER(), table=<samples>), Column('BB_959', INTEGER(), table=<samples>), Column('BB_960', INTEGER(), table=<samples>), Column('BB_961', INTEGER(), table=<samples>), Column('

In [175]:
res = db.session.query(samples)
print(res[0].keys())

['otu_id', 'BB_940', 'BB_941', 'BB_943', 'BB_944', 'BB_945', 'BB_946', 'BB_947', 'BB_948', 'BB_949', 'BB_950', 'BB_952', 'BB_953', 'BB_954', 'BB_955', 'BB_956', 'BB_958', 'BB_959', 'BB_960', 'BB_961', 'BB_962', 'BB_963', 'BB_964', 'BB_966', 'BB_967', 'BB_968', 'BB_969', 'BB_970', 'BB_971', 'BB_972', 'BB_973', 'BB_974', 'BB_975', 'BB_978', 'BB_1233', 'BB_1234', 'BB_1235', 'BB_1236', 'BB_1237', 'BB_1238', 'BB_1242', 'BB_1243', 'BB_1246', 'BB_1253', 'BB_1254', 'BB_1258', 'BB_1259', 'BB_1260', 'BB_1264', 'BB_1265', 'BB_1273', 'BB_1275', 'BB_1276', 'BB_1277', 'BB_1278', 'BB_1279', 'BB_1280', 'BB_1281', 'BB_1282', 'BB_1283', 'BB_1284', 'BB_1285', 'BB_1286', 'BB_1287', 'BB_1288', 'BB_1289', 'BB_1290', 'BB_1291', 'BB_1292', 'BB_1293', 'BB_1294', 'BB_1295', 'BB_1296', 'BB_1297', 'BB_1298', 'BB_1308', 'BB_1309', 'BB_1310', 'BB_1374', 'BB_1415', 'BB_1439', 'BB_1441', 'BB_1443', 'BB_1486', 'BB_1487', 'BB_1489', 'BB_1490', 'BB_1491', 'BB_1494', 'BB_1495', 'BB_1497', 'BB_1499', 'BB_1500', 'BB_1501',

In [72]:
# successful in getting data using inspector
inspector = inspect(db.engine)

In [73]:
print(inspector.get_table_names())

['otu', 'samples', 'samples_metadata']


In [82]:
print(inspector.get_columns('samples'))

AttributeError: 'Inspector' object has no attribute 'get_column'

In [79]:
print(metadata.tables)

immutabledict({})


In [118]:
# 
# testing for /out_description
# 

In [ ]:
# defining metatdata
metadata = MetaData()
metadata.reflect(bind=db.engine)
# getting the samples table     
otu = metadata.tables['otu']
res = db.session.query(otu)
descriptions = []
for r in res:
    descriptions.append(r.lowest_taxonomic_unit_found)
descriptions

In [ ]:
# 
# testing for /otu/<sample>
# 

In [152]:
sample = "940"
metadata = MetaData()
metadata.reflect(bind=db.engine)
# getting the samples table     
samples_metadata = metadata.tables['samples_metadata']
res = db.session.query(samples_metadata).filter_by(SAMPLEID=sample).first()
data = {
    "AGE": res.AGE,
    "BBTYPE": res.BBTYPE,
    "ETHNICITY": res.ETHNICITY,
    "GENDER": res.GENDER,
    "LOCATION": res.LOCATION,
    "SAMPLEID": res.SAMPLEID
}

print(data)

{'AGE': 24, 'BBTYPE': 'I', 'ETHNICITY': 'Caucasian', 'GENDER': 'F', 'LOCATION': 'Beaufort/NC', 'SAMPLEID': 940}


In [ ]:
# 
# testing for /wfreq/<sample>
# 

In [153]:
sample = "940"
metadata = MetaData()
metadata.reflect(bind=db.engine)
# getting the samples table     
samples_metadata = metadata.tables['samples_metadata']
res = db.session.query(samples_metadata).filter_by(SAMPLEID=sample).first()
res.WFREQ

2

In [ ]:
# 
# testing for /samples/<sample>
# 

In [174]:
sample = "BB_940"
df = pd.read_sql_table('samples', "sqlite:///Datasets/belly_button_biodiversity.sqlite")
df = df.sort_values(sample, ascending=False)
df[sample].tolist()
data = [
    {
        otu_ids : df['otu_id'].tolist()
    },
    {
        sample_values: df[sample].tolist()
    }
]

[163,
 126,
 113,
 78,
 71,
 51,
 50,
 47,
 40,
 40,
 37,
 36,
 30,
 28,
 25,
 23,
 22,
 19,
 19,
 14,
 13,
 13,
 13,
 12,
 12,
 11,
 11,
 11,
 10,
 10,
 10,
 8,
 7,
 7,
 7,
 6,
 5,
 5,
 5,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0

# TESTING END

# ACTUAL FLASK APP

In [124]:
@app.route("/")
def home():
    return render_template("index.html")

In [ ]:
@app.route("/names")
def get_otu_ids():
    # defining metatdata
    metadata = MetaData()
    metadata.reflect(bind=db.engine)
    # getting the samples table     
    samples = metadata.tables['samples']
    # converting table into something we can actually use  
    res = db.session.query(samples)
    return res[0].keys()

In [ ]:
@app.route("/otu")
def get_descriptions():
    # defining metatdata
    metadata = MetaData()
    metadata.reflect(bind=db.engine)
    # getting the samples table     
    otu = metadata.tables['otu']
    res = db.session.query(otu)
    descriptions = []
    for r in res:
        descriptions.append(r.lowest_taxonomic_unit_found)
    return descriptions

In [ ]:
@app.route("/metadata/<sample>")
def get_sample_info(sample):
    sample = sample.replace("BB_", "")
    metadata = MetaData()
    metadata.reflect(bind=db.engine)
    # getting the samples table     
    samples_metadata = metadata.tables['samples_metadata']
    res = db.session.query(samples_metadata).filter_by(SAMPLEID=sample).first()
    data = {
        "AGE": res.AGE,
        "BBTYPE": res.BBTYPE,
        "ETHNICITY": res.ETHNICITY,
        "GENDER": res.GENDER,
        "LOCATION": res.LOCATION,
        "SAMPLEID": res.SAMPLEID
    }
    
    return data

In [ ]:
@app.route('/wfreq/<sample>')
def get_washing_freq(sample):
    sample = sample.replace("BB_", "")
    metadata = MetaData()
    metadata.reflect(bind=db.engine)
    # getting the samples table     
    samples_metadata = metadata.tables['samples_metadata']
    res = db.session.query(samples_metadata).filter_by(SAMPLEID=sample).first()
    return res.WFREQ

In [ ]:
@app.route('/samples/<sample>')
def get_filtered_sample_values:
    sample = "BB_940"
    df = pd.read_sql_table('samples', "sqlite:///Datasets/belly_button_biodiversity.sqlite")
    df = df.sort_values(sample, ascending=False)
    data = [
        {
            otu_ids : df['otu_id'].tolist()
        },
        {
            sample_values: df[sample].tolist()
        }
    ]
    
    return data